In [1]:
import model as M
import evaluate as E
import config as CFG
import os
import torch

In [2]:
DATASETS = ["billsum", "cnn_dailymail", "newsroom", "big_patent", "scientific_papers"]

In [3]:
def train(hierarchical, loss_func, suffix='', separate=False):
    for DATASET in DATASETS:
        train_set = M.CustomDataset(os.path.join(CFG.DATASET_ROOT, DATASET, CFG.METHOD, 'train.tsv'), hierarchical=hierarchical)
        print(len(train_set))

        model = M.Siamese(separate)
        model.to(CFG.DEVICE)

        print("Training from", DATASET)
        M.train_model(model, train_set, loss_func=loss_func, shuffle=False)

        CKPT_PATH = os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "model.pth")
        if not os.path.exists(os.path.dirname(CKPT_PATH)):
            os.makedirs(os.path.dirname(CKPT_PATH))

        scorer = model.base_model
        torch.save(scorer.state_dict(), CKPT_PATH)

        del model
        torch.cuda.empty_cache()

In [4]:
def test(suffix='', separate=False):
    for DATASET in DATASETS:
        CKPT_PATH = os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "model.pth")
        scorer = M.Scorer(separate)
        scorer.load_state_dict(torch.load(CKPT_PATH, map_location=CFG.DEVICE))
        scorer.to(CFG.DEVICE)
        scorer.eval()

        E.evaluate_newsroom("human/newsroom/newsroom-human-eval.csv", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_newsroom.tsv"), scorer)
        E.evaluate_realsumm("human/realsumm/realsumm_100.tsv", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_realsumm.tsv"), scorer)
        E.evaluate_tac("human/tac/TAC2010_all.json", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_tac.tsv"), scorer)

        del scorer
        torch.cuda.empty_cache()

In [ ]:
train(hierarchical=False, loss_func='CrossEntropyLoss', separate=False)
test(separate=False)
# train(hierarchical=False, loss_func='MarginRankingLoss', suffix='_NonHier', separate=True)
# test(suffix='_NonHier', separate=True)

Hierarchichal False
64152


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training from billsum
CrossEntropyLoss 0.0 False


  0%|▌                                                                                                                                                                                       | 50/15000 [00:39<3:17:13,  1.26it/s]